**The bot versions of the Skype group**

*Born from nothing by the genius, Eric, using only his notebooks and some of Google's GPUs, the bots have arrived. Each mirrors its owner to some infinitesimal degree*

Special Awards:

*    David-bot: The most coherent of the bunch

*    Diego-bot: The most mopey

*    Miles-bot & Gwyn-bot: Incoherent babbling about one another

*    Milo-bot: Spent the most time in school, just like the real one

*    Jett-bot: Talks most about videogames

*    Parsa-bot: A massive D&D nerd

*    Mars-bot: Offensive, bordering on actionable

*    Eric-bot: So hard to replicate that the genius-bot actually killed all attempts at being trained

*Disclaimer. Anything generated by the bots is solely on the AI and really only reflects on the friend whose data was used to train it, not on me*

**IMPORTANT INSTRUCTIONS**

Steps to run:

1. Download and install git (google it)

2. Open your terminal or command prompt and type "git clone https://github.com/Leuyangh/quoteAIs.git"

3. Open the downloaded folder and find the "FinalModels" folder

4. To the left of this notebook in Google colab, open up the file explorer and click "Mount google drive"

5. Upload the FinalModels folder and make sure it is under "My Drive" and not a subfolder. Path should look like drive->My Drive->FinalModels

6. Go to Runtime at the top here and click "run all"

7. Find the bottom cell with `generateQuote("MILO", 20, 10)` in it. That's the call that generates quotes. Either add new cells and run it with different names or just modify that call. Shift+Enter to run just that cell and save some time. 

8. Addendums 

> `generateQuote("NAME" - name of bot to use, 10 - number here is how long the quote should be in words, 10 - number of quotes to generate)`

>e.g. `generateQuote("MARS", 50, 5)` will generate 5 quotes from Mars, each 50 words long

>Alternatively, use `generateSentence(name)` to get a sentence from the Ngram model (statistical generation not a recurrent neural net) 
>>This only returns a string of the sentence generated, to print you need to use print(generateSentence(name))

To update if I ever create a new model:

1. Go into the directory made when you git cloned the repo

2. run "git pull"

3. Upload that new folder to drive

**Code, just ignore**

In [25]:
import pickle
import numpy as np
import string, os, random
import warnings
import h5py
import datetime

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from numpy import loadtxt
from tensorflow.keras.models import load_model

from numpy.random import seed

tensorflow.random.set_seed(int(datetime.datetime.utcnow().timestamp()))
seed(int(datetime.datetime.utcnow().timestamp()))

In [26]:
def loadTokenizer(name):
    name = "/content/drive/My Drive/FinalModels/" + name + "_tokenizer.pkl"
    tokenizer = pickle.load(open(name, 'rb'))
    return tokenizer

def loadFromFile(filename):
    file = open(filename, 'r', encoding = "utf-8")
    text = file.read()
    file.close()
    return text

def load_quotegen(name):
    return pickle.load(open("/content/drive/My Drive/FinalModels/" + name + '.pkl', 'rb') )

In [27]:
def generateQuote(name = 'PARSA', length = 30, count = 5):
    in_filename = "/content/drive/My Drive/FinalModels/" + name + "_sequences.txt"
    file = loadFromFile(in_filename)
    lines = file.split('\n')
    tokenizer = loadTokenizer(name)
    sequences = tokenizer.texts_to_sequences(lines)
    sequences = np.array(sequences)
    X, y = sequences[:,:-1], sequences[:,-1]
    seq_length = X.shape[1]
    model = load_model("/content/drive/My Drive/FinalModels/" + name + '_model.h5')
    for i in range(0, count):
        seed_text = lines[np.random.randint(0,len(lines))]
        result = list()
        for _ in range(length):
            # encode the text as integer
            encoded = tokenizer.texts_to_sequences([seed_text])[0]
            # truncate sequences to a fixed length
            encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
            # predict probabilities for each word
            yhat = np.argmax(model.predict(encoded), axis = -1)
            # map predicted word index to word
            out_word = ''
            for word, index in tokenizer.word_index.items():
                if index == yhat:
                    out_word = word
                    break
            # append to input
            seed_text += ' ' + out_word
            result.append(out_word)
        final = ' '.join(result)
        print(f'{name}: "{final}"')

def generateQuoteSeeded(name = 'PARSA', input = ""):
    tokenizer = loadTokenizer(name)
    model = load_model("/content/drive/My Drive/FinalModels/" + name + '_model.h5')
    result = list()
    lines = input.split()
    for _ in range(len(input)):
            # encode the text as integer
            encoded = tokenizer.texts_to_sequences([lines])[0]
            # truncate sequences to a fixed length
            encoded = pad_sequences([encoded], maxlen=50, truncating='pre')
            # predict probabilities for each word
            yhat = np.argmax(model.predict(encoded), axis = -1)
            # map predicted word index to word
            out_word = ''
            for word, index in tokenizer.word_index.items():
                if index == yhat:
                    out_word = word
                    break
            # append to input
            lines += ' ' + out_word
            result.append(out_word)
    final = ' '.join(result)
    print(f'{name}: "{final}"')

def generate_sentence(name):
    quote_gen = load_quotegen(name)
    text = [None,None,None]
    sentence_finished = False
    while not sentence_finished:
        r = random.random()
        accumulator = .0

        for word in quote_gen[tuple(text[-3:])].keys():
            accumulator += quote_gen[tuple(text[-3:])][word]

            if accumulator >= r:
                text.append(word)
                break

        if text[-3:] == [None,None,None]:
            sentence_finished = True

    return ' '.join([t for t in text if t])

**Make calls here using instructions above**

Shift+enter to run a cell. If any errors pop up make sure your drive is mounted and the folder is in there correctly. If any runtime errors occur try running all the previous cells with "Runtime"->run all or shift+enter in them one by one.

Names should be capitalized

In [9]:
names = ["DAVID", "DIEGO", "GWYN", "JETT", "MARS", "MILES", "MILO", "PARSA"] #ERIC to come when I figure it out

*Recommended that you set the second parameter to something long, like 20-50 at least and then pick out your favorite sentence from within it. Bot is likely to start generating based on text partway through a message so you want it long enough that the bot translates from the end of that message all the way through the end of the one after at least, likely will end with a few words into the next-next message or even next-next-next message*

In [28]:
generateQuote("MILO", 20, 10)

MILO: "back in her reverted form i want a new champ and i think it was just a good change i"
MILO: "who spoke little english told me that exact thing about me me ill change much like minutes ago we need"
MILO: "a lunch with my phone on kro so i can make you to chillin on two seperate mustve no structure"
MILO: "holy shit dude they dont get it at am good so i think sunday is poseidon kinda good skirmishers simulator"
MILO: "its actually good just not wrong they dont understand right ik actually im saying yea i am sick xd and"
MILO: "good yea yea fosho with one better team need to be deep idiot i think a chick gemis food and"
MILO: "system flips a coin look at how this guy you favoring trumps inaction means he didnt get how it is"
MILO: "a new league i could fit at but if its a good idea if you can do it poorly i"
MILO: "in the pot not the end of the world you one what if there is that if it fails this"
MILO: "something have you been at games of a young guy but its just a little worse it

In [29]:
for i in range(0, 10):
  print("Parsa: " + generate_sentence("PARSA"))

Parsa: are you claiming that the lack of emotion
Parsa: we have two people who belong to order
Parsa: Bs it 's not racist
Parsa: sitting on samecord btw
Parsa: im back in santa monica
Parsa: How about a Jedi trained by dumbledore to act as the starship enterprise ’ s chief mechanic but ended up falling to the ruinous powers of chaos ?
Parsa: ummm
Parsa: I WANT ONE
Parsa: is n't it just dark kaer morhen
Parsa: oh hey


In [30]:
for n in names:
  generateQuote(n, 20, 1)

DAVID: "nightmare on elm street issa nightmare on elm street a nightmare on elm street feel like jason friday hockey mask"
DIEGO: "second mask edited previous message edge edgs i got this one too black is the new orange even more if"
GWYN: "culture for a couple glass beads africa from define cultural appropriation in such a retarded manner taking elements of someones"
JETT: "speaking of stupidity and food one of the likely reasons for their primitive brains is the fact that additionally to"
MARS: "most wrong is not a good idea and you can basing it happen for a bunch of negusians or negusites"
MILES: "see what youre typing michael so ill assume youre typing color because i cant see itmiles obrien today pm miles"
MILO: "we love and it had to use it then no need to slouch on etiquette tho ah should be me"
PARSA: "secure alliances to fight off a godlike entity in a neighboring island and an even more likely event is that"


This is just a thing I tried to do where you can feed a bot some sample thing and it'll convert to {person}'s version of it but I don't think its working right yet

In [31]:
generateQuoteSeeded("MILES", "hi hi hi")

MILES: "do n naysh o m o t i"


**Supplemental Info**

GenerateQuote takes some time to start up but is quick to perform multiple predictions due to the overhead of loading in the files. Will try to fix that but it may be inevitable. To work around this, use the third parameter to generate a bunch of quotes at once

The cell starting with "for n in names" is REALLY slow so don't hit run-all unless you want that to run. Just use shift-enter on the cell you do want